
### 1. 问题分析

Kaggle 竞赛： [SanFrancisco Crime](https://www.kaggle.com/c/sf-crime/data)  
问题描述：  
从 1934 年到 1963 年，旧金山因在逃往恶魔岛的途中关押了一批世界上最臭名昭著的罪犯而臭名昭著。   
如今，这座城市以其科技产业而非犯罪历史而闻名。然而，随着贫富差距加剧、住房短缺，以及乘坐湾区捷运上班的昂贵电子玩具泛滥，这座海湾城市的犯罪率也随之上升。  
本次比赛的数据集涵盖了旧金山所有街区近 12 年的犯罪报告，涵盖从日落区到南加州大学，从滨海区到埃克塞尔西奥区。给定时间和地点，你需要预测发生的犯罪类型。   

多分类问题：  
输入特征： 地点，时间，周几 等
目标变量：犯罪类型

#### 核心思路
- 准备数据
- 使用HDBSCAN 聚类，生成新特征--cluster表示经纬度聚类（比经纬度更适宜XGB训练）
- 使用XGBoost 训练模型--多分类问题
- 预测和评估
- 提交Kaggle：评估模型预测准确率还行，对test测试集预测，将预测结果转化成kaggle submission规定的格式输出



In [1]:
# dataset_basedir： 请先将数据集下载解压后放到指定目录并设置 dataset_basedir
dataset_basedir = "/Users/jimmy/Resources/AI Models/dataset" 

### 2. 数据加载与分析

In [2]:
!pip install scikit-learn==1.5.2

import pandas as pd

base_dir = dataset_basedir + '/sf-crime'
test_df = pd.read_csv(base_dir+'/test.csv')
train_df = pd.read_csv(base_dir+'/train.csv')
print(test_df.shape, test_df.columns)
print(train_df.shape, train_df.columns)
train_df.head(2)

(884262, 7) Index(['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y'], dtype='object')
(878049, 9) Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'],
      dtype='object')


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599


In [3]:
train_df.X.dtypes, train_df.Y.dtypes 

(dtype('float64'), dtype('float64'))

### 3. 数据处理

用x,y经纬度做HDBSCAN，聚类

In [4]:
import hdbscan
from hdbscan import approximate_predict

min_samples = None
min_cluster_size = 10
hdb = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric='euclidean', prediction_data=True)  # 加上这个！！)
coordinate_list = train_df[['X','Y']]
train_df['Cluster'] = hdb.fit_predict(coordinate_list) # 相关于训练，预测时使用approximate_predict
train_df.head(2)

/opt/homebrew/anaconda3/envs/colab/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/homebrew/anaconda3/envs/colab/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Cluster
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,9264
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,9264


In [5]:
train_df.Cluster.value_counts()

Cluster
 11918    26354
-1        13791
 7793      4449
 14393     3891
 1313      3170
          ...  
 4060        10
 11453       10
 8301        10
 670         10
 10347       10
Name: count, Length: 17453, dtype: int64

注意： Cluster=-1的行是噪音行

In [6]:
print(train_df.shape)
train_df = train_df[train_df.Cluster!=-1]
train_df.shape

(878049, 10)


(864258, 10)

In [7]:
#one-hot将object字段转数值
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

In [8]:
onehot = OneHotEncoder(sparse_output=False, drop='first')
onehot_cols = ['DayOfWeek']
onehot_array= onehot.fit_transform(train_df[onehot_cols])
week_df = pd.DataFrame(onehot_array, columns=onehot.get_feature_names_out(onehot_cols),index=train_df.index)
week_df.shape

(864258, 6)

注：加入 index = train_df.index 这个参数，把train_df.index的索引复制到新的dataFrame结构上 来保证处理后的矩阵的行与原矩阵的行能一一对应上

In [9]:
dates = pd.to_datetime(train_df.Dates).astype('int')/10**9
dates_df = pd.DataFrame(dates, columns=['Dates'],index=train_df.index)
dates_df.shape 

(864258, 1)

In [10]:
# 把策略转成数值
label_encoder = LabelEncoder()
ctg = label_encoder.fit_transform(train_df['Category'])
ctg_df = pd.DataFrame(ctg,index=train_df.index)
ctg_df.shape 

(864258, 1)

### 4. 模型训练

使用下列特征训练：
- 犯罪时间戳 -周期性特征
- 周几 -onehot
- 经纬度聚类

In [11]:
cluster_df = pd.DataFrame(train_df['Cluster'], index=train_df.index)
cluster_df.shape

(864258, 1)

In [12]:
X = pd.concat([cluster_df, dates, week_df], axis=1)
y = ctg_df
X

,Cluster,Dates,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
0,9264,1.431561e+09,0.0,0.0,0.0,0.0,0.0,1.0
1,9264,1.431561e+09,0.0,0.0,0.0,0.0,0.0,1.0
2,5289,1.431560e+09,0.0,0.0,0.0,0.0,0.0,1.0
3,5164,1.431560e+09,0.0,0.0,0.0,0.0,0.0,1.0
4,7121,1.431560e+09,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
878042,2857,1.041812e+09,1.0,0.0,0.0,0.0,0.0,0.0
878043,2857,1.041812e+09,1.0,0.0,0.0,0.0,0.0,0.0
878044,4442,1.041812e+09,1.0,0.0,0.0,0.0,0.0,0.0
878046,10901,1.041811e+09,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X.dtypes

Cluster                  int64
Dates                  float64
DayOfWeek_Monday       float64
DayOfWeek_Saturday     float64
DayOfWeek_Sunday       float64
DayOfWeek_Thursday     float64
DayOfWeek_Tuesday      float64
DayOfWeek_Wednesday    float64
dtype: object

In [14]:
print(X.isnull().sum())

Cluster                0
Dates                  0
DayOfWeek_Monday       0
DayOfWeek_Saturday     0
DayOfWeek_Sunday       0
DayOfWeek_Thursday     0
DayOfWeek_Tuesday      0
DayOfWeek_Wednesday    0
dtype: int64


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.2,random_state=32)

In [27]:
X_test

,Cluster,Dates,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday
739542,7612,1.100171e+09,0.0,0.0,0.0,1.0,0.0,0.0
789912,11351,1.078602e+09,0.0,1.0,0.0,0.0,0.0,0.0
579926,2598,1.172428e+09,0.0,0.0,1.0,0.0,0.0,0.0
36302,8119,1.416066e+09,0.0,1.0,0.0,0.0,0.0,0.0
184029,13344,1.354307e+09,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
736396,10996,1.101499e+09,0.0,0.0,0.0,0.0,0.0,0.0
476559,10458,1.219609e+09,0.0,0.0,1.0,0.0,0.0,0.0
822418,2597,1.064948e+09,0.0,0.0,0.0,0.0,1.0,0.0
173214,10345,1.359142e+09,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=100, random_state=42,enable_categorical=True)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### 5. 模型评估

In [36]:
y_pred = xgb.predict(X_test)

In [37]:
from sklearn.metrics import mean_squared_error, r2_score
mse_xgb = mean_squared_error(y_test, y_pred)
r2_xgb = r2_score(y_test, y_pred)
mse_xgb, r2_xgb

(140.78002568671465, -0.23685240845446565)

不用不再往下进行了， 模型预测效果太差了

In [16]:
# 定义一个数据转化工具:将原数据特征 转化为 训练和预测 XGB的特征
from hdbscan import approximate_predict

def convert_data_to_xgb_X(train_df):
    onehot_cols = ['DayOfWeek']
    onehot_array= onehot.fit_transform(train_df[onehot_cols])
    week_df = pd.DataFrame(onehot_array, columns=onehot.get_feature_names_out(onehot_cols),index=train_df.index)
    
    # 
    cluster = approximate_predict(hdb,train_df[['X','Y']])
    cluster_df = pd.DataFrame(cluster,index=train_df.index)
    
    dates = pd.to_datetime(train_df.Dates).astype('int')/10**9
    dates_df = pd.DataFrame(dates, columns=['Dates'],index=train_df.index)
    
    return pd.concat([dates_df, week_df, cluster_df], axis=1)
    

In [17]:
label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
label_mapping

{0: 'ARSON',
 1: 'ASSAULT',
 2: 'BAD CHECKS',
 3: 'BRIBERY',
 4: 'BURGLARY',
 5: 'DISORDERLY CONDUCT',
 6: 'DRIVING UNDER THE INFLUENCE',
 7: 'DRUG/NARCOTIC',
 8: 'DRUNKENNESS',
 9: 'EMBEZZLEMENT',
 10: 'EXTORTION',
 11: 'FAMILY OFFENSES',
 12: 'FORGERY/COUNTERFEITING',
 13: 'FRAUD',
 14: 'GAMBLING',
 15: 'KIDNAPPING',
 16: 'LARCENY/THEFT',
 17: 'LIQUOR LAWS',
 18: 'LOITERING',
 19: 'MISSING PERSON',
 20: 'NON-CRIMINAL',
 21: 'OTHER OFFENSES',
 22: 'PORNOGRAPHY/OBSCENE MAT',
 23: 'PROSTITUTION',
 24: 'RECOVERED VEHICLE',
 25: 'ROBBERY',
 26: 'RUNAWAY',
 27: 'SECONDARY CODES',
 28: 'SEX OFFENSES FORCIBLE',
 29: 'SEX OFFENSES NON FORCIBLE',
 30: 'STOLEN PROPERTY',
 31: 'SUICIDE',
 32: 'SUSPICIOUS OCC',
 33: 'TREA',
 34: 'TRESPASS',
 35: 'VANDALISM',
 36: 'VEHICLE THEFT',
 37: 'WARRANTS',
 38: 'WEAPON LAWS'}

In [19]:
test_df

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
...,...,...,...,...,...,...,...
884257,884257,2003-01-01 00:01:00,Wednesday,MISSION,2600 Block of BRYANT ST,-122.408983,37.751987
884258,884258,2003-01-01 00:01:00,Wednesday,NORTHERN,1900 Block of WASHINGTON ST,-122.425342,37.792681
884259,884259,2003-01-01 00:01:00,Wednesday,INGLESIDE,5500 Block of MISSION ST,-122.445418,37.712075
884260,884260,2003-01-01 00:01:00,Wednesday,BAYVIEW,1500 Block of HUDSON AV,-122.387394,37.739479
